# Homework 4

## Problem 1: Random $k$-bit Prime

Remember that an integer $n$ has $k$ bits if $2^{k-1} \leq n < 2^k$.  In this problem, you need to give a *random* $k$-bit prime for a given $k$.

**Note:**  This is a one-liner!  You *can* use Sage's built in function `random_prime`.  To get a prime between `A` and `B`, *including both*, you do `random_prime(B,lbound=A)`.

In [ ]:
def rand_kbit_prime(k):
    """
    Given k, finds a random k-bit prime.

    INPUT:
    k: number of bits (positive integer).

    OUTPUT:
    A k-bit prime.
    """
    ...

Test it:

In [ ]:
number_of_tests = 30
mink = 10
maxk = 20

for _ in range(number_of_tests):
    k = randint(mink, maxk + 1)
    p = rand_kbit_prime(k)
    if not is_prime(p):
        print(f"The output {p = } is not prime.")
        break
    n_bits = floor(log(p, base=2)) + 1
    if n_bits != k:
        print(f"The output {p = } does not have {k} bits, it has {n_bits} bits.")
        break
else:
    print("It works!")

## Problem 2: Naive Discrete Log

Given $g$ and $h$ in a $\mathbb{F}_p^{\times}$, finds the *discrete log* $\log_g(h)$, i.e., $x$ such that $g^x = h$.

Here you should just "brute force it", i.e., compute $g^2$, $g^3$, etc., until you find a match.

This is very simple, but there is one catch: there might not be such $x$, in which case your algorithm should say so!  **Let's convention that in this case, the output should be $-1$.**

You can do this is as follows:

1) If $h=1$, return $x=0$.
2) If not, compute the powers $g$, $g^2$, $g^3$ until either we get $h$ or we get $1$.
3) If we get $1$ before $h$, there is no such $x$.  (Can you see why?)  So, return $-1$.  If not, return the power that gave $h$.

**Important:** You **have** to compute the powers efficiently, as described in the section [Computing Successive Powers](https://luisfinotti.org/pcimc/05-Powers.html#successive-powers) in the [book](https://luisfinotti.org/pcimc/).  *Points will be taken if you do not follow it!*

**Note:** The inputs $h$ and $g$ will be assumed to already be elements in $\mathbb{F}_p^{\times}$ and not integers.

In [ ]:
def ndl(h, g):
    """
    Given h and g in a multiplicative group, finds the discrete log of h base g
    by brute force.

    INPUTS:
    h, g: units of Zmod(p).

    OUTPUT:
    The discrete log base g of h, i.e., the power x such that g^x = h (an integer
    between 0 and the order of g minus one).  If no such power exists, returns -1.
    """
    ...

Testing (using Sage's own `discrete_log` function!):

In [ ]:
# test when the log exists
number_of_tests = 20
minp = 20
maxp = 1000

for _ in range(number_of_tests):
    p = random_prime(maxp, lbound=minp)
    F = FiniteField(p)
    h = F(randint(1, p - 1))
    g = F.primitive_element() # a primitive element
    result = ndl(h, g)
    expected = discrete_log(h, g)
    if result != expected:
        print(f"It fails for {p = }, {g = }, and {h = }.  It gives {result} when it should give {expected}.")
        break
else:
    print("It works when the discrete log exits!")


# test when the log does not exist
number_of_tests = 5
minp = 20
maxp = 1000

for _ in range(number_of_tests):
    p = random_prime(maxp, lbound=minp)
    F = FiniteField(p)
    h = F.primitive_element()
    g = h^2
    result = ndl(h, g)
    if result != -1:
        print(f"It fails for {p = }, {g = }, {h = }.  It gives {result} when it should give -1.")
        break
else:
    print("It works when the discrete log doesn't exit!")

## Problem 3: Elgamal Cryptosystem

In this problem you will create *two* functions: one to encode and one to decode messages using the Elgamal cryptosystem.

For the *encoding* function you are given a generator (`g`), a public key (`A`) in $\mathbb{F}_p$ (remember that $A = g^a \in \mathbb{F}_p$, where $a$ is the *private* key) and a message (`m`), which will assume is also in $\mathbb{F}_p$, and should output a *pair* `[c1, c2]` (as [described in the text](https://luisfinotti.org/pcimc/06-DH_and_ElGamal.html#steps-for-elgamal-encryption)).

**Notes:**
- You are not given $p$ explicitly, but, as before, you can find it using `parent(A).characteristic()` if you need it.
- If you need $|g|$ (the order of $g$), you can use `g.multiplicative_order()`.

In [ ]:
def elgamal_enc(g, A, m):
    """
    Elgamal encryption with generator g, public key A, and message m.

    INPUTS:
    g: the generator, a unit of Zmod(p) (for some prime p) of large prime order.
    A: the public key (also unit of Zmod(p)).
    m: the message  (also unit of Zmod(p)).

    OUTPUT:
    The encrypted message using ElGamal's cryptosystem (a list with two units in
    Zmod(p)).
    """
    ...

Now we need a decode function: given a generator (`g`) a secret key (`a`) and an encoded message (that used the public key $A = g^a$) `enc_message`, where `enc_message = [c1, c2]` (so, the input is a list with two entries), it should output the (original) decoded message `m`.

In [ ]:
def elgamal_dec(g, a, enc_message):
    """
    Decrypts the Elgamal encrypted message enc_message, with generator g and
    private key a.

    INPUTS:
    g: the generator, a unit of Zmod(p) (for some prime p) of large prime order.
    a: the private key (an integer between 2 and p-2).
    enc_message: the encoded message (a list with two units of Zmod(p)).

    OUTPUT:
    The original decoded message.
    """
    ...

To test this, we will use *your* encryption and decryption functions.  I will check if those are correct, but note that, although unlikely, it could be possible you have the wrong methods and still pass the test below.

In [ ]:
number_of_tests = 30
k = 20  # number of bits

for _ in range(number_of_tests):
    while True:  # find p and q
        q = random_prime(2^k - 1, lbound=2^(k - 1))
        p = 2 * q + 1  # want (p-1)/2 to be prime
        if is_prime(p):
            break
    g = FiniteField(p).primitive_element()^2
    a = randint(floor(q / 4), q - 2)
    A = g^a
    m = randint(2, p - 2)
    enc_message = elgamal_enc(g, A, m)
    result = elgamal_dec(g, a, enc_message)
    if result != m:
        print(
            f"It fails for {p = }, {g = }, and {a = }.  When sending {m = }, encryption/decryption gives {result}"
        )
        break
else:
    print("It works!")

## Problem 4: Babystep-Giantstep Algorithm

Implement the [Shank's Babystep-Giantstep Algorithm](https://luisfinotti.org/pcimc/07-Computing_DL.html#al-bs_gs) described in our book to compute discrete logs.  The input/output are the same as from Problem 2: $h$ and $g$, and the output is $x$ such that $g^x=h$.

As before, if there discrete log does not exist, return $-1$.

The [example in the book](https://luisfinotti.org/pcimc/07-Computing_DL.html#baby-step-giant-step) should be quite helpful here!  Most of the work is done there and you just need to adapt the code to your function.

**Important:** The output should be an *integer* between $0$ and $|g| - 1$.

In [ ]:
def bsgs(h, g):
    """
    Given h and g in a multiplicative group, finds the discrete log of h base
    using Shank's Babystep-Giantstep algorithm.

    INPUTS:
    h, g: units of Zmod(p).

    OUTPUT:
    The discrete log base g of h, i.e., the power x such that g^x = h (an integer
    between 0 and the order of g minus one).  If no such power exists, returns -1.
    """
    ...

Testing:

In [ ]:
# test when the log exists
number_of_tests = 20
minp = 20
maxp = 1000

for _ in range(number_of_tests):
    p = random_prime(maxp, lbound=minp)
    F = FiniteField(p)
    h = F(randint(1, p - 1))
    g = F.primitive_element() # a primitive element
    result = bsgs(h, g)
    expected = discrete_log(h, g)
    if result != expected:
        print(f"It fails for {p = }, {g = }, and {h = }.  It gives {result} when it should give {expected}.")
        break
else:
    print("It works when the discrete log exits!")


# test when the log does not exist
number_of_tests = 5
minp = 20
maxp = 1000

for _ in range(number_of_tests):
    p = random_prime(maxp, lbound=minp)
    F = FiniteField(p)
    h = F.primitive_element()
    g = h^2
    result = bsgs(h, g)
    if result != -1:
        print(f"It fails for {p = }, {g = }, {h = }.  It gives {result} when it should give -1.")
        break
else:
    print("It works when the discrete log doesn't exit!")

### Optional: Comparison

If you want, you can compare the efficiency of the algrithms by evaluating the cells below:

In [ ]:
k = 23
p = random_prime(2^(k+1), lbound=2^k)
F = FiniteField(p)
g = F.primitive_element()
x = (p - 1)/2
h = g^x

Naive:

In [ ]:
%%time
x = ndl(h,g)

Babystep-Giantstep:

In [ ]:
%%time
x = bsgs(h,g)

Sage's own, of course, is faster!

In [ ]:
%%time
x = discrete_log(h,g)